In [2]:
import os

temp_localfolder = '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary/'

# List directories
files = os.listdir(temp_localfolder)

In [3]:
# filter jobs
jobs = list(filter(lambda x: x.startswith('MD_Test_TEBEG'),files))
n_jobs = len(jobs)

In [11]:
import pandas as pd
import py4vasp as pv
import numpy as np
import re 

data = pd.DataFrame({
    'Energy [eV]': 1000*[0],
    'Step': np.linspace(1,1000, 1000)
})

datapoints = []

for job in jobs:
    calc = pv.Calculation.from_path(f'{temp_localfolder}{job}')
    entry = calc.energy[:].read( "ETOTAL, temperature")
    tebeg = re.search('TEBEG_(.*)k_3', job)
    tebeg = tebeg.group(1)

    datapoints.append(
        pd.DataFrame({
            'Energy [eV]': entry['total energy   ETOTAL'],
            'Temperature [K]': entry['temperature    TEIN'],
            'Step': np.linspace(1,1000,1000),
            'TEBEG [K]': tebeg
        })
    )

results = pd.concat(datapoints)

In [24]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from colorhash import ColorHash

# graphing 1x2
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for setting in results['TEBEG [K]'].unique():
    subset = results.query('`TEBEG [K]`==@setting')
    fig.append_trace(go.Scatter(
                x=subset['Step'],
                y=subset['Energy [eV]'],
                name=f'TEBEG [K]: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
                legendgroup=f'{setting}',
                marker={'color':ColorHash(setting).hex}
            ),
            row=1, col=1
            )
    fig.append_trace(go.Scatter(
                x=subset['Step'],
                y=subset['Temperature [K]'],
                name=f'TEBEG [K]: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
                legendgroup=f'{setting}',
                showlegend=False,
                marker={'color':ColorHash(setting).hex}
            ),
            row=2, col=1
            )
fig.update_layout(
        title_text = 'TEBEG testing',
        )

fig.update_xaxes(title_text='Step', row=2, col=1)
fig.update_yaxes(title_text='Energy [eV]', row=1, col=1)
fig.update_yaxes(title_text='Temperature [K]', row=2, col=1)

In [25]:
# save plot as HTML file
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/TEBEG_testing.html')